# 入门 Prompt Flow

**先决条件** - 要充分利用本教程，您需要：
- Prompt Flow repo 的本地克隆
- 支持 Jupyter Notebook 的 Python 环境（例如 Jupyter Lab 或 Visual Studio Code 的 Python 扩展）
- 知道如何使用 Python 编程

_具有机器学习的基本理解可能会有所帮助，但不是必需的。_

**学习目标** - 完成本教程后，您应该能够：

- 运行您的第一个 Prompt Flow 示例
- 运行您的第一个评估

本教程中使用的示例是 [web-classification](../../flows/standard/web-classification/README.md) 流，它将 URL 分类为几个预定义类别。分类是传统的机器学习任务，此示例说明如何使用 GPT 和提示执行分类。


## 0. 安装依赖库

In [ ]:
%pip install -r ../../requirements.txt

## 1. 创建必要的连接
连接有助于安全地存储和管理与 LLM 和其他外部工具交互所需的秘密密钥或其他敏感凭据，例如 Azure 内容安全性。

在这个笔记本中，我们将使用流 `web-classification`，它在内部使用连接 `open_ai_connection`，如果我们之前没有添加它，我们需要设置连接。创建后，它将存储在本地数据库中，并可以在任何流中使用。

如果您未曾创建过OpenAI资源，请按照此[说明](https://learn.microsoft.com/en-us/azure/cognitive-services/openai/how-to/create-resource?pivots=web-portal)准备您的 Azure Open AI 资源，并获取您的 `api_key`。

如果您已经创建过OpenAI资源，请直接在您的Azure Portal中获取`api_base`和`api_key`。


In [2]:
import json
from promptflow import PFClient
from promptflow.connections import AzureOpenAIConnection, OpenAIConnection

# client can help manage your runs and connections.
pf = PFClient()

注意：这里要将前面准备好的OpenAI资源的`api_base`和`api_key`填入以下代码中，从而创建一个OpenAI连接。`api_version`请默认填入`2023-05-15`。

In [3]:
try:
    conn_name = "open_ai_connection"
    conn = pf.connections.get(name=conn_name)
    print("using existing connection")
except:
    # Follow https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/create-resource?pivots=web-portal to create an Azure Open AI resource.
    connection = AzureOpenAIConnection(
        name=conn_name,
        api_key="<your_openai_key>", #ToDO replace with your own key
        api_base="<your_openai_base>", #ToDO replace with your own base
        api_type="azure",
        api_version="<test_version>", #ToDO replace with the version you want to use
    )

    # use this if you have an existing OpenAI account
    # connection = OpenAIConnection(
    #     name=conn_name,
    #     api_key="<user-input>",
    # )

    conn = pf.connections.create_or_update(connection)
    print("successfully created connection")

print(conn)

using existing connection
name: open_ai_connection
module: promptflow.connections
created_date: '2023-08-31T12:16:58.884737'
last_modified_date: '2023-08-31T12:16:58.884737'
type: azure_open_ai
api_key: '******'
api_base: https://aoaijapaneast01.openai.azure.com/
api_type: azure
api_version: '2023-05-15'



## 2. 运行 web-classification 流

`web-classification` 是一个演示使用 LLM 进行多类分类的流。给定一个 URL，它将使用简单的摘要和分类提示将 URL 分类为一个 Web 类别。



### 设置flow的地址

In [4]:
flow = "../../flows/standard/web-classification"  # path to the flow directory

### 测试

In [5]:
# Test flow
flow_inputs = {
    "url": "https://play.google.com/store/apps/details?id=com.twitter.android",
}
flow_result = pf.test(flow=flow, inputs=flow_inputs)
print(f"Flow result: {flow_result}")

2023-08-31 15:50:18 +0800   13748 execution          INFO     Start to run 5 nodes with concurrency level 16.
2023-08-31 15:50:18 +0800   13748 execution.flow     INFO     Executing node fetch_text_content_from_url. node run id: 7a0928f3-43f0-4a11-87b5-3436cbd9b7c1_fetch_text_content_from_url_0
2023-08-31 15:50:18 +0800   13748 execution.flow     INFO     Executing node prepare_examples. node run id: 7a0928f3-43f0-4a11-87b5-3436cbd9b7c1_prepare_examples_0
2023-08-31 15:50:19 +0800   13748 execution.flow     INFO     Node prepare_examples completes.
2023-08-31 15:50:19 +0800   13748 execution.flow     INFO     Node fetch_text_content_from_url completes.
2023-08-31 15:50:19 +0800   13748 execution.flow     INFO     Executing node summarize_text_content. node run id: 7a0928f3-43f0-4a11-87b5-3436cbd9b7c1_summarize_text_content_0
2023-08-31 15:50:24 +0800   13748 execution.flow     INFO     Node summarize_text_content completes.
2023-08-31 15:50:24 +0800   13748 execution.flow     INFO     

In [6]:
# Test single node in the flow
node_name = "fetch_text_content_from_url"
node_inputs = {
    "url": "https://play.google.com/store/apps/details?id=com.twitter.android"
}
flow_result = pf.test(flow=flow, inputs=node_inputs, node=node_name)
print(f"Node result: {flow_result}")

2023-08-31 15:50:26 +0800   13748 execution.flow     INFO     Executing node fetch_text_content_from_url. node run id: d3484623-f104-485c-8feb-b2cf56b7e8e1_fetch_text_content_from_url_c971ce0d-9d4d-4066-9c34-9042ea2b0864
2023-08-31 15:50:26 +0800   13748 execution.flow     INFO     Node fetch_text_content_from_url completes.
Node result: X - Apps on Google Playgoogle_logo PlayGamesAppsMovies & TVBooksKidsnonesearchhelp_outline Sign in with Googleplay_appsLibrary & devicespaymentPayments & subscriptionsreviewsMy Play activityredeemOffersPlay PasssettingsSettingsPrivacy Policy • Terms of ServiceGamesAppsMovies & TVBooksKidsXX Corp.Contains adsIn-app purchases3.9star21.3M reviews1B+DownloadsMature 17+infoInstallShareAdd to wishlistAbout this apparrow_forwardThe X app is the trusted global digital town square for everyone.With X, you can:- Post content for the world to see and join public conversations- Stay up to date on breaking news and follow your interests- Stay better informed with e

### 使用Data file中的多行数据进行批量测试


In [7]:
data = "../../flows/standard/web-classification/data.jsonl"  # path to the data file

# create run with default variant
base_run = pf.run(flow=flow, data=data, stream=True)

2023-08-31 15:50:31 +0800   25316 execution          INFO     Start to run 5 nodes with concurrency level 2.
2023-08-31 15:50:31 +0800   25316 execution.flow     INFO     Executing node fetch_text_content_from_url. node run id: c_users_wazhi_documents_github_fy24devhack_promptflow_examples_flows_standard_web_classification_default_20230831_155027_193152_fetch_text_content_from_url_1
2023-08-31 15:50:31 +0800   25316 execution.flow     INFO     Executing node prepare_examples. node run id: c_users_wazhi_documents_github_fy24devhack_promptflow_examples_flows_standard_web_classification_default_20230831_155027_193152_prepare_examples_1
2023-08-31 15:50:31 +0800    7752 execution          INFO     Start to run 5 nodes with concurrency level 2.
2023-08-31 15:50:31 +0800    7752 execution.flow     INFO     Executing node fetch_text_content_from_url. node run id: c_users_wazhi_documents_github_fy24devhack_promptflow_examples_flows_standard_web_classification_default_20230831_155027_193152_fet

In [8]:
details = pf.get_details(base_run)
details.head(10)

,inputs.url,inputs.line_number,outputs.category,outputs.evidence
0,https://www.youtube.com/watch?v=kYqRtjDBci8,0,None,None
1,https://arxiv.org/abs/2307.04767,1,Academic,Both
2,https://play.google.com/store/apps/details?id=...,2,App,Both


## 3. 评估您的流程
然后，您可以使用评估方法来评估您的流程。评估方法也是flow，它们使用 Python 或 LLM 等计算指标，如准确性、相关性分数。

在这个笔记本中，我们使用 `classification-accuracy-eval` 流程进行评估。这是一个演示如何评估分类系统性能的流程。它涉及将每个预测与基准进行比较，并分配“正确”或“不正确”的等级，并聚合结果以产生诸如准确性之类的指标，反映系统在对数据进行分类方面的表现如何。


### 在上一批次运行上运行评估
**base_run** 是我们在上面的第 2 步中完成的批处理运行，实现以 "data.jsonl" 作为输入的 Web 分类流程。


In [9]:
eval_flow = "../../flows/evaluation/eval-classification-accuracy"

eval_run = pf.run(
    flow=eval_flow,
    data="../../flows/standard/web-classification/data.jsonl",  # path to the data file
    run=base_run,  # specify base_run as the run you want to evaluate
    column_mapping={
        "groundtruth": "${data.answer}",
        "prediction": "${run.outputs.category}",
    },  # map the url field from the data to the url input of the flow
    stream=True,
)

2023-08-31 15:50:42 +0800    2100 execution          INFO     Start to run 1 nodes with concurrency level 2.
2023-08-31 15:50:42 +0800    2100 execution.flow     INFO     Executing node grade. node run id: c_users_wazhi_documents_github_fy24devhack_promptflow_examples_flows_evaluation_eval_classification_accuracy_default_20230831_155039_236049_grade_2
2023-08-31 15:50:42 +0800   45676 execution          INFO     Start to run 1 nodes with concurrency level 2.
2023-08-31 15:50:42 +0800   45676 execution.flow     INFO     Executing node grade. node run id: c_users_wazhi_documents_github_fy24devhack_promptflow_examples_flows_evaluation_eval_classification_accuracy_default_20230831_155039_236049_grade_0
2023-08-31 15:50:42 +0800   21068 execution          INFO     Start to run 1 nodes with concurrency level 2.
2023-08-31 15:50:42 +0800   21068 execution.flow     INFO     Executing node grade. node run id: c_users_wazhi_documents_github_fy24devhack_promptflow_examples_flows_evaluation_eval_c

In [10]:
details = pf.get_details(eval_run)
details.head(10)

,inputs.groundtruth,inputs.prediction,inputs.line_number,outputs.grade
0,Channel,None,0,Incorrect
1,Academic,Academic,1,Correct
2,App,App,2,Correct


In [11]:
metrics = pf.get_metrics(eval_run)
print(json.dumps(metrics, indent=4))

{
    "accuracy": 0.67
}


In [12]:
pf.visualize([base_run, eval_run])

The HTML file is generated at 'C:\\Users\\wazhi\\AppData\\Local\\Temp\\pf-visualize-detail-sxq3il0_.html'.
Trying to view the result in a web browser...
Successfully visualized from the web browser.


到目前为止，您已经成功运行了第一个Prompt Flow，甚至对其进行了评估。太棒了！

您可以查看[web-classifcation](../../flows/standard/web-classification/)流和[classification-accuracy](../../flows/evaluation/eval-classification-accuracy/)流以获取更多详细信息，并开始构建自己的流程。

或者您可以继续进行更进阶的主题：尝试使用变体。


### 使用变体的批量测试

[变体（Variant）](../../../docs/concepts/concept-variants.md) 能够帮助你进行基于LLM的优化迭代。你可以设置一个变体并使用不同的参数（例如温度），或者不同的Prompt来测试批量数据下的效果提升。

在接下来的示例中， `web-classification`中的node `summarize_text_content` 有两个不同的变体: `variant_0` and `variant_1`. 他们的差异来自于参数上的不同：

variant_0:

    - inputs:
        - deployment_name: gpt-35-turbo
        - max_tokens: '128'
        - temperature: '0.2'
        - text: ${fetch_text_content_from_url.output}

variant_1:

    - inputs:
        - deployment_name: gpt-35-turbo
        - max_tokens: '256'
        - temperature: '0.3'
        - text: ${fetch_text_content_from_url.output}


你可以参考完整的flow定义在文件 [flow.dag.yaml](../../flows/standard/web-classification/flow.dag.yaml)中。

In [13]:
# use the variant1 of the summarize_text_content node.
variant_run = pf.run(
    flow=flow,
    data=data,
    variant="${summarize_text_content.variant_1}",  # here we specify node "summarize_text_content" to use variant 1 version.
    stream=True,
)

2023-08-31 15:50:58 +0800   44888 execution          INFO     Start to run 5 nodes with concurrency level 2.
2023-08-31 15:50:58 +0800   26240 execution          INFO     Start to run 5 nodes with concurrency level 2.
2023-08-31 15:50:58 +0800   44888 execution.flow     INFO     Executing node fetch_text_content_from_url. node run id: c_users_wazhi_documents_github_fy24devhack_promptflow_examples_flows_standard_web_classification_variant_1_20230831_155054_373449_fetch_text_content_from_url_0
2023-08-31 15:50:58 +0800   44888 execution.flow     INFO     Executing node prepare_examples. node run id: c_users_wazhi_documents_github_fy24devhack_promptflow_examples_flows_standard_web_classification_variant_1_20230831_155054_373449_prepare_examples_0
t_content_from_url_1
2023-08-31 15:50:58 +0800   26240 execution.flow     INFO     Executing node prepare_examples. node run id: c_users_wazhi_documents_github_fy24devhack_promptflow_examples_flows_standard_web_classification_variant_1_20230831_1

In [14]:
details = pf.get_details(variant_run)
details.head(10)

,inputs.url,inputs.line_number,outputs.category,outputs.evidence
0,https://www.youtube.com/watch?v=kYqRtjDBci8,0,None,None
1,https://arxiv.org/abs/2307.04767,1,Academic,Text content
2,https://play.google.com/store/apps/details?id=...,2,App,Both


### 评估不同变体的效果
后续我们可以比较不同变体下的metrics来看哪个效果更好。

In [15]:
eval_flow = "../../flows/evaluation/eval-classification-accuracy"

eval_run_variant = pf.run(
    flow=eval_flow,
    data="../../flows/standard/web-classification/data.jsonl",  # path to the data file
    run=variant_run,  # use run as the variant
    column_mapping={
        "groundtruth": "${data.answer}",
        "prediction": "${run.outputs.category}",
    },  # map the url field from the data to the url input of the flow
    stream=True,
)

2023-08-31 15:51:13 +0800   40644 execution          INFO     Start to run 1 nodes with concurrency level 2.
2023-08-31 15:51:13 +0800   40644 execution.flow     INFO     Executing node grade. node run id: c_users_wazhi_documents_github_fy24devhack_promptflow_examples_flows_evaluation_eval_classification_accuracy_default_20230831_155110_122598_grade_0
2023-08-31 15:51:13 +0800   26048 execution          INFO     Start to run 1 nodes with concurrency level 2.
2023-08-31 15:51:13 +0800   26048 execution.flow     INFO     Executing node grade. node run id: c_users_wazhi_documents_github_fy24devhack_promptflow_examples_flows_evaluation_eval_classification_accuracy_default_20230831_155110_122598_grade_1
2023-08-31 15:51:13 +0800    4776 execution          INFO     Start to run 1 nodes with concurrency level 2.
2023-08-31 15:51:13 +0800    4776 execution.flow     INFO     Executing node grade. node run id: c_users_wazhi_documents_github_fy24devhack_promptflow_examples_flows_evaluation_eval_c

In [16]:
details = pf.get_details(eval_run_variant)
details.head(10)

,inputs.groundtruth,inputs.prediction,inputs.line_number,outputs.grade
0,Channel,None,0,Incorrect
1,Academic,Academic,1,Correct
2,App,App,2,Correct


In [17]:
metrics = pf.get_metrics(eval_run_variant)
print(json.dumps(metrics, indent=4))

{
    "accuracy": 0.67
}


In [18]:
pf.visualize([eval_run, eval_run_variant])

The HTML file is generated at 'C:\\Users\\wazhi\\AppData\\Local\\Temp\\pf-visualize-detail-ae5081eo.html'.
Trying to view the result in a web browser...
Successfully visualized from the web browser.


# 下一步

后续你还可以参考以下进阶教程:
- [Manage connections](../../connections/connection.ipynb): how to manage the endpoints/secrets information to access external services including LLMs.
- [Chat with PDF](../e2e-development/chat-with-pdf.md): go through an end-to-end tutorial on how to develop a chat application with prompt flow.
- [Deploy http endpoint](../flow-deploy/deploy.md): how to deploy the flow as a local http endpoint.
- [Prompt flow in Azure AI](./quickstart-azure.ipynb): run and evaluate flow in Azure AI where you can collaborate with team better.